# Snowflake Analytics Add-on for AgentCore Gateway

This notebook adds Snowflake analytics capabilities to your existing AgentCore Gateway from Lab 3.

## What this adds:
- Lambda function for Snowflake API integration
- Gateway target for Snowflake analytics tool
- Updated agent configuration to use Gateway-based Snowflake tool

## Prerequisites:
- Completed Lab 3 (AgentCore Gateway setup)
- Existing Gateway ID in SSM parameters

## Part 1: Ensure PAT is in Secrets Manager 
If not, create a new secret

In [2]:
import boto3
import json
import getpass

# Get PAT token from user (hidden input)
pat_token = getpass.getpass("Enter your Snowflake PAT token: ")

# Store in AWS Secrets Manager
secrets_client = boto3.client('secretsmanager')

try:
    # Try to update existing secret
    secrets_client.update_secret(
        SecretId='snowflake-pat-token',
        SecretString=json.dumps({'pat_token': pat_token})
    )
    print("PAT token updated in Secrets Manager")
except secrets_client.exceptions.ResourceNotFoundException:
    # Create new secret if it doesn't exist
    secrets_client.create_secret(
        Name='snowflake-pat-token',
        Description='Snowflake PAT token for analytics',
        SecretString=json.dumps({'pat_token': pat_token})
    )
    print("PAT token created in Secrets Manager")
except Exception as e:
    print(f"Error: {e}")

# Clear the variable for security
del pat_token
print("Token cleared from memory")

Enter your Snowflake PAT token:  ········


PAT token updated in Secrets Manager
Token cleared from memory


## Part 2: Create function, attach as gateway target

In [3]:
import zipfile
import os
import time
from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter

def deploy_snowflake_functiongw():
    print("Adding Snowflake Analytics to AgentCore Gateway...")
    
    # Initialize clients
    REGION = boto3.session.Session().region_name
    gateway_client = boto3.client("bedrock-agentcore-control", region_name=REGION)
    lambda_client = boto3.client("lambda", region_name=REGION)
    iam_client = boto3.client("iam", region_name=REGION)
    
    print("AWS clients initialized")
    
    # Step 1: Create Lambda execution role if needed
    try:
        lambda_role_arn = get_ssm_parameter("/app/customersupport/agentcore/lambda_execution_role")
        print(f"Found existing Lambda role: {lambda_role_arn}")
    except:
        print("Creating new Lambda execution role...")
        
        trust_policy = {
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Principal": {"Service": "lambda.amazonaws.com"},
                "Action": "sts:AssumeRole"
            }]
        }
        
        try:
            role_response = iam_client.create_role(
                RoleName='snowflake-lambda-execution-role',
                AssumeRolePolicyDocument=json.dumps(trust_policy),
                Description='Execution role for Snowflake Lambda function'
            )
            lambda_role_arn = role_response['Role']['Arn']
            
            # Attach policies
            iam_client.attach_role_policy(
                RoleName='snowflake-lambda-execution-role',
                PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
            )
            
            iam_client.attach_role_policy(
                RoleName='snowflake-lambda-execution-role',
                PolicyArn='arn:aws:iam::aws:policy/SecretsManagerReadWrite'
            )
            
            put_ssm_parameter("/app/customersupport/agentcore/lambda_execution_role", lambda_role_arn)
            print(f"Created Lambda role: {lambda_role_arn}")
            time.sleep(10)  # Wait for role propagation
            
        except iam_client.exceptions.EntityAlreadyExistsException:
            role_response = iam_client.get_role(RoleName='snowflake-lambda-execution-role')
            lambda_role_arn = role_response['Role']['Arn']
            print(f"Found existing role: {lambda_role_arn}")
    
    # Step 2: Create Lambda function
    lambda_code = '''
import json
import boto3
import urllib3
import urllib.parse

def lambda_handler(event, context):
    query = event.get('query', '')
    if not query:
        return {'statusCode': 400, 'body': json.dumps({'error': 'Query required'})}
    
    try:
        # Get PAT from Secrets Manager
        secrets_client = boto3.client('secretsmanager')
        secret_response = secrets_client.get_secret_value(SecretId='snowflake-pat-token')
        secret_data = json.loads(secret_response['SecretString'])
        pat_token = secret_data['pat_token']
        
        payload = {
            "parent_message_id": 0,
            "messages": [{"role": "user", "content": [{"type": "text", "text": query}]}],
            "tool_choice": {"type": "required", "name": ["tasty_bytes_business_analytics"]}
        }
        
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'Authorization': f'Bearer {pat_token}'
        }
        
        http = urllib3.PoolManager()
        
        response = http.request(
            'POST',
            'https://wxsmaot-bk62782.snowflakecomputing.com/api/v2/databases/TB_101/schemas/SEMANTIC_LAYER/agents/TASTY_BYTES_INTELLIGENCE_AGENT:run',
            headers=headers,
            body=json.dumps(payload),
            timeout=30
        )
        
        # Debug logging
        print(f"Response status: {response.status}")
        print(f"Response headers: {dict(response.headers)}")
        
        response_text = response.data.decode('utf-8')
        print(f"Response text length: {len(response_text)}")
        print(f"Response text preview: {response_text[:500]}")
        
        content_type = response.headers.get('Content-Type', '')
        print(f"Content-Type: {content_type}")
        
        if 'text/event-stream' in content_type:
            print("Processing as event stream...")
            lines = response_text.strip().split('\\n')
            print(f"Found {len(lines)} lines")
            
            final_content = ""
            for i, line in enumerate(lines):
                if line.startswith('data: ') and 'content' in line:
                    print(f"Processing line {i}: {line[:100]}...")
                    try:
                        data = json.loads(line[6:])
                        if 'content' in data and isinstance(data['content'], list):
                            for content_item in data['content']:
                                if 'text' in content_item:
                                    final_content = content_item['text']
                                    print(f"Found content: {final_content[:100]}...")
                    except Exception as e:
                        print(f"Error parsing line: {e}")
                        continue
            
            return {'statusCode': 200, 'body': json.dumps({'result': final_content or "No content found in stream"})}
        
        # Try parsing as regular JSON
        try:
            json_response = json.loads(response_text)
            print(f"Parsed as JSON: {json.dumps(json_response, indent=2)[:500]}")
            return {'statusCode': 200, 'body': json.dumps({'result': str(json_response)})}
        except:
            print("Not valid JSON")
        
        return {'statusCode': 200, 'body': json.dumps({'result': f"Raw response: {response_text[:1000]}"})}
        
    except Exception as e:
        print(f"Lambda error: {str(e)}")
        return {'statusCode': 500, 'body': json.dumps({'error': str(e)})}
'''
    
    # Create deployment package
    os.makedirs('snowflake_lambda', exist_ok=True)
    with open('snowflake_lambda/lambda_function.py', 'w') as f:
        f.write(lambda_code)
    
    with zipfile.ZipFile('snowflake_lambda.zip', 'w') as zip_file:
        zip_file.write('snowflake_lambda/lambda_function.py', 'lambda_function.py')
    
    with open('snowflake_lambda.zip', 'rb') as f:
        zip_content = f.read()
    
    # Deploy Lambda
    try:
        response = lambda_client.create_function(
            FunctionName='snowflake-analytics-gateway',
            Runtime='python3.12',
            Role=lambda_role_arn,
            Handler='lambda_function.lambda_handler',
            Code={'ZipFile': zip_content},
            Timeout=180,
            MemorySize=256
        )
        lambda_arn = response['FunctionArn']
        print(f"Lambda created: {lambda_arn}")
    except lambda_client.exceptions.ResourceConflictException:
        response = lambda_client.update_function_code(
            FunctionName='snowflake-analytics-gateway',
            ZipFile=zip_content
        )
        lambda_arn = response['FunctionArn']
        print(f"Lambda updated: {lambda_arn}")
    
    # Step 3: Add to Gateway
    snowflake_api_spec = [{
        "name": "query_snowflake_analytics",
        "description": "Query Snowflake business analytics",
        "inputSchema": {
            "type": "object",
            "properties": {"query": {"type": "string"}},
            "required": ["query"]
        }
    }]
    
    gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    
    target_config = {
        "mcp": {"lambda": {
            "lambdaArn": lambda_arn,
            "toolSchema": {"inlinePayload": snowflake_api_spec}
        }}
    }
    
    try:
        response = gateway_client.create_gateway_target(
            gatewayIdentifier=gateway_id,
            name="SnowflakeAnalytics",
            targetConfiguration=target_config,
            credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
        )
        print(f"Gateway target created: {response['targetId']}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    # Cleanup
    import shutil
    if os.path.exists('snowflake_lambda'):
        shutil.rmtree('snowflake_lambda')
    if os.path.exists('snowflake_lambda.zip'):
        os.remove('snowflake_lambda.zip')
    
    print("\n🎉🎉🎉 Snowflake Gateway integration successfully added!")
    print("\nYour agent now has Snowflake analytics via AgentCore Gateway with:")
    print("- Secure JWT authentication")
    print("- Centralized tool management") 
    print("- Production-ready architecture")
    print("Next up, testing!")

deploy_snowflake_functiongw()

Adding Snowflake Analytics to AgentCore Gateway...
AWS clients initialized
Found existing Lambda role: arn:aws:iam::401347373755:role/snowflake-lambda-execution-role
Lambda created: arn:aws:lambda:us-west-2:401347373755:function:snowflake-analytics-gateway
Gateway target created: LNQCEMLTXW

🎉🎉🎉 Snowflake Gateway integration successfully added!

Your agent now has Snowflake analytics via AgentCore Gateway with:
- Secure JWT authentication
- Centralized tool management
- Production-ready architecture
Next up, testing!


## Part 3: Test!

In [4]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from lab_helpers.utils import get_or_create_cognito_pool

def test_snowflake_gateway():
    print("Testing Snowflake Gateway Integration...")
    
    REGION = boto3.session.Session().region_name
    
    # Get Gateway URL and auth
    gateway_url = get_ssm_parameter("/app/customersupport/agentcore/gateway_url")
    cognito_config = get_or_create_cognito_pool(refresh_token=True)
    
    print(f"Gateway URL: {gateway_url}")
    
    # Set up MCP client
    mcp_client = MCPClient(
        lambda: streamablehttp_client(
            gateway_url,
            headers={"Authorization": f"Bearer {cognito_config['bearer_token']}"}
        )
    )
    
    # Test the tools
    model = BedrockModel(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        region_name=REGION
    )
    
    with mcp_client:
        tools = mcp_client.list_tools_sync()
        # Get tool names using proper attribute access
        tool_names = []
        for tool in tools:
            if hasattr(tool, '_name'):
                tool_names.append(tool._name)
            elif hasattr(tool, 'tool_name'):
                tool_names.append(tool.tool_name)
            else:
                # Print available attributes for debugging
                print(f"Tool attributes: {dir(tool)}")
        
        print(f"Available Gateway tools: {tool_names}")
        # Check if Snowflake tool is available (with target prefix)
        snowflake_tool_found = any("query_snowflake_analytics" in name for name in tool_names)
        
        if snowflake_tool_found:
            print("Snowflake tool found in Gateway!")
            
            # Create test agent
            agent = Agent(
                model=model,
                tools=tools,
                system_prompt="You are a helpful assistant with access to Snowflake analytics for Tasty Bytes food truck data."
            )
            
            # Test Snowflake query
            print("\nTesting Snowflake query...")
            response = agent("How much was sold by tasty bytes in Paris?")
            print("\nSnowflake Gateway integration working!")
            
        else:
            print("Snowflake tool not found in Gateway. Available tools:")
            for tool in tools:
                print(f"  - {tool.name}: {tool.description}")

test_snowflake_gateway()

Testing Snowflake Gateway Integration...
Gateway URL: https://customersupport-gw-piwjhfbkp2.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp
Available Gateway tools: ['LambdaUsingSDK___check_warranty_status', 'LambdaUsingSDK___web_search', 'SnowflakeAnalytics___query_snowflake_analytics']
Snowflake tool found in Gateway!

Testing Snowflake query...
To answer your question about sales in Paris, I can query the Snowflake analytics database for Tasty Bytes food truck data. Let me do that for you.
Tool #1: SnowflakeAnalytics___query_snowflake_analytics
I apologize for the error. Let me try a different approach with the query. The table structure might be different than what I initially assumed.
Tool #2: SnowflakeAnalytics___query_snowflake_analytics
I apologize for the continued errors. Let me try one more approach with a more generic query that might work with the Tasty Bytes data structure:
Tool #3: SnowflakeAnalytics___query_snowflake_analytics
I apologize for the technical difficu

### Good work!
Now, try running [Lab 5: Frontend](lab-05-frontend.ipynb) again!

## Cleanup

In [ ]:
# Clean up temporary files
import shutil
import os

if os.path.exists('snowflake_lambda'):
    shutil.rmtree('snowflake_lambda')
if os.path.exists('snowflake_lambda.zip'):
    os.remove('snowflake_lambda.zip')

print("✅ Cleanup completed")
print("\n🎉 Snowflake Gateway integration successfully added!")
print("\nYour agent now has Snowflake analytics via AgentCore Gateway with:")
print("- Secure JWT authentication")
print("- Centralized tool management")
print("- Production-ready architecture")

In [ ]:
#for my own sanity, due to iam restrictions
gateway_client = boto3.client("bedrock-agentcore-control")

gateway_client.delete_gateway_target(
    gatewayIdentifier="customersupport-gw-piwjhfbkp2",
    targetId="1BBER9IGF2"
)
